In [1]:
# SparkSession 생성하기
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [2]:
spark

In [3]:
# range 명령을 통해 DataFrame 생성하면 기본적으로 8개의 파티션이 생성됨
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
# 파티션 재분배
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
# 조인 수행, spark.sql.shuffle.partitions 속성의 기본값이 2000이라서 200으로 설정됨
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect()

[Row(sum(id)=2500000000000)]

In [4]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#8L)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#66]
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
      +- *(6) Project [id#8L]
         +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
            :- *(3) Sort [id#8L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [id=#50]
            :     +- *(2) Project [(id#0L * 5) AS id#8L]
            :        +- Exchange RoundRobinPartitioning(5), REPARTITION_WITH_NUM, [id=#46]
            :           +- *(1) Range (2, 10000000, step=2, splits=4)
            +- *(5) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [id=#57]
                  +- Exchange RoundRobinPartitioning(6), REPARTITION_WITH_NUM, [id=#56]
                     +- *(4) Range (2, 10000000, step=4, splits=4)




In [7]:
df1.printSchema()

root
 |-- id: long (nullable = false)

